In [9]:
# ######### data mungling with pandas before app-py for Flask # ######### 

In [3]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import datetime as dt

In [11]:
invDemo=pd.read_csv("static/coloradoTestItemsR0_bed1.csv", encoding='ISO-8859-1')

testCol= invDemo.columns

for col in testCol:
    print(invDemo[col])

0     0
1     0
2     0
3     0
4     0
5     0
6     0
7     0
8     0
9     0
10    0
11    0
12    0
13    0
14    0
15    0
Name: roundNo, dtype: int64
0     Colorado
1     Colorado
2     Colorado
3     Colorado
4     Colorado
5     Colorado
6     Colorado
7     Colorado
8     Colorado
9     Colorado
10    Colorado
11    Colorado
12    Colorado
13    Colorado
14    Colorado
15    Colorado
Name: State, dtype: object
0     one- bedroom
1     one- bedroom
2     one- bedroom
3     one- bedroom
4     one- bedroom
5     one- bedroom
6     one- bedroom
7     one- bedroom
8     one- bedroom
9     one- bedroom
10    one- bedroom
11    one- bedroom
12    one- bedroom
13    one- bedroom
14    one- bedroom
15    one- bedroom
Name: Type, dtype: object
0     13593125.80
1     13905571.31
2     14685543.29
3     13990524.37
4     14397781.31
5     13622982.97
6     13598032.94
7     13954719.70
8     13630323.67
9     14722779.07
10    14647467.67
11    14313046.22
12    13958801.66
13    1459224

In [14]:
# R-0 here means round zero not r-sqr B-1 bedroom
fileTestR0B1 = 'static/coloradoTestItemsR0_bed1.csv'
testColoradoR0B1_df = pd.read_csv(fileTestR0B1, encoding='ISO-8859-1')
testColoradoR0B1_df.head()

,roundNo,State,Type,itemPrice,itemIconURL,itemDesc,medianPrice,regPol2,regLinFit,regLinFree
0,0,Colorado,one- bedroom,13593125.80,static/img/1-4.png,desc,150283.3333,146192.85,137117,124473
1,0,Colorado,one- bedroom,13905571.31,static/img/1-5.png,desc,150283.3333,146192.85,137117,124473
2,0,Colorado,one- bedroom,14685543.29,static/img/1-4.png,desc,150283.3333,146192.85,137117,124473
3,0,Colorado,one- bedroom,13990524.37,static/img/1-3.png,desc,150283.3333,146192.85,137117,124473
4,0,Colorado,one- bedroom,14397781.31,static/img/1-4.png,desc,150283.3333,146192.85,137117,124473


In [18]:
fileTest = 'static/coloradoTestData.csv'

In [19]:
coTestRaw_df = pd.read_csv(fileTest)
coTestRaw_df.head()

,roundNo,ind3x,DateYear,DateHalf,1bedAvg,2bedAvg,3bedAvg,4bedAvg,avgOneBedStr,avgTwoBedStr,avgThreeBedStr,avgCondoStr
0,1,2010-Q1,2010,1,150283.3333,174066.6667,228088.7500,182990.5833,"$150,283","$174,067","$228,089","$182,991"
1,2,2010-Q2,2010,2,150583.3333,169433.3333,220449.8333,170216.6667,"$150,583","$169,433","$220,450","$170,217"
2,3,2011-Q1,2011,1,141525.0000,153400.0000,197003.7500,164754.1667,"$141,525","$153,400","$197,004","$164,754"
3,4,2011-Q2,2011,2,144291.6667,156316.6667,202016.6667,180133.3333,"$144,292","$156,317","$202,017","$180,133"
4,5,2012-Q1,2012,1,148633.3333,153351.1667,198033.3333,180566.6667,"$148,633","$153,351","$198,033","$180,567"


In [13]:
coTest_df = coTestRaw_df.copy()
# coTest_df.columns ['roundNo', 'ind3x', 'DateYear', 'DateHalf', '1bedAvg', '2bedAvg',
#        '3bedAvg', '4bedAvg', 'avgOneBedStr', 'avgTwoBedStr', 'avgThreeBedStr',
#        'avgCondoStr']

In [14]:
# - - - - - REG FOR ONE BEDROOM MEDIAN LST PRC - - - - - 
# pol reg order 2 eqtn y = 620.55x2 - 2167.7x + 147740 highest R-sq
# linear reg w/ forced intercept lower R-sq
# lin reg free style high R-sq but unrealistic intercept at 116,000

In [20]:
fileTest2 = 'static/coloradoTestReg_bed1.csv'
coTestBed1_df = pd.read_csv(fileTest2)
coTestBed1_df.head()

,roundNo,1bedAvg,1bedAvgRnd,regPol2,regLinFit,regLinFree,1bedAvg_str,1bedAvgRnd_str,regPol2_str,regLinFit_str,regLinFree_str
0,1,150283.3333,179376,146192.85,137117,124473,"$ 150,283","$ 179,376","$ 146,193","$ 137,117","$ 124,473"
1,2,150583.3333,145546,145886.80,144234,132854,"$ 150,583","$ 145,546","$ 145,887","$ 144,234","$ 132,854"
2,3,141525.0000,145212,146821.85,151351,141235,"$ 141,525","$ 145,212","$ 146,822","$ 151,351","$ 141,235"
3,4,144291.6667,174960,148998.00,158468,149616,"$ 144,292","$ 174,960","$ 148,998","$ 158,468","$ 149,616"
4,5,148633.3333,172193,152415.25,165585,157997,"$ 148,633","$ 172,193","$ 152,415","$ 165,585","$ 157,997"
